# Spatial Correspondance Using Ray Casting of Ischemic and Territory Regions on the Surface of the Myocardium

Begin by specifying the path to the directory that contains the data required for ray-based analysis. This directory must include the following five files:

 1. MBF Model (VTU format): A volumetric model extracted from the MBF (Myocardial Blood Flow) map.

 2. Basal Slice (VTP format): A slice of the MBF model captured around the base of the myocardium.

 3. Apical Slice (VTP format): A slice of the MBF model representing the apical hemisphere.

 4. Ischemic Region (VTU format): A segmentation of the ischemic region associated with the vessel of interest, also extracted from the MBF map.

 5. MBF Territories Labels (dat format): A text file including the name tag and the number tag of the territories assigned to each branch.

Also, the extracted surface of the MBF map model should be saved into the parent directory of the vessel specific working directory.

After providing the input files, configure the TerritoryTags. These tags indicate myocardial regions downstream of a stenosis, based on the output file MBF_Territories.vtu.

Finally, ensure that the image scale is set appropriately—either in centimeters (cm) or millimeters (mm)—to match the spacing of the input image data. Accurate scaling is essential for proper geometric and spatial interpretation.

In [15]:
InputFolder = "/Users/ana/Documents/AnahitaSeresti/05_PrePostCABG/RayCasting/SU04A"
slice_apex = f"{InputFolder}/Slice_Apex.vtp"
slice_base = f"{InputFolder}/Slice_Base.vtp"
InputMyocardium = f"{InputFolder}/Myocardium.vtp"
InputMBF = f"{InputFolder}/MBF_Territories.vtu"
InputLV_binarylabelmap = f"{InputFolder}/LV.vtk"


scale = 'cm'

In [16]:
import os
import vtk
import numpy as np
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk
from utilities import ReadVTUFile, ReadVTPFile, WriteVTPFile, WriteVTUFile, GetCentroid, ThresholdPointsByUpper, LargestConnectedRegion, PrintProgress, ThresholdByUpper, ExtractSurface, ReadVTKFile

NRaySection = 1500
NRaySphere = 40000
NSection = 100


if scale == 'cm':
    Radius = 3.5
    L_Section_Ray = 10
    L_Sphere_Ray = 5
    Ray_Thickness = 0.02
    Base_Cover = 3
    Apex_Cover = 0.3

elif scale == 'mm':
    Radius = 35
    L_Section_Ray = 100
    L_Sphere_Ray = 50
    Ray_Thickness = 0.2
    Base_Cover = 10
    Apex_Cover = 3

else:
    print("provide a proper scale")



### Defining necessary functions

In [3]:

def Line(point1, point2, res):
    line = vtk.vtkLineSource()
    line.SetPoint1(point1)
    line.SetPoint2(point2)
    line.SetResolution(res)
    line.Update()
    
    return line.GetOutput()

def BoldLine(centerline):
    tube_filter = vtk.vtkTubeFilter()
    tube_filter.SetInputData(centerline)
    tube_filter.SetRadius(Ray_Thickness)  # Adjust this value to change thickness
    tube_filter.SetNumberOfSides(50)  # Higher = smoother tube
    tube_filter.CappingOn()  # Close tube ends
    tube_filter.Update()
    
    return tube_filter.GetOutput()

def SliceWPlane(Volume,Origin,Norm):
    plane=vtk.vtkPlane()
    plane.SetOrigin(Origin)
    plane.SetNormal(Norm)
    Slice=vtk.vtkCutter()
    Slice.GenerateTrianglesOff()
    Slice.SetCutFunction(plane)
    Slice.SetInputData(Volume)
    Slice.Update()
    
    return Slice.GetOutput()

def ClipSurfaceWPlane(surface, center, axis, IO = False):
    plane = vtk.vtkPlane()
    plane.SetOrigin(center)
    plane.SetNormal(axis)

    clipper = vtk.vtkClipPolyData()
    clipper.SetInputData(surface)
    clipper.SetClipFunction(plane)
    clipper.GenerateClippedOutputOff()
    if IO:
        clipper.InsideOutOn()
    else:
        clipper.InsideOutOff()
    #clipper.GetOutputInformation(1)
    clipper.Update()

    return clipper.GetOutput()

def ClipVolumeWPlane(volume, center, axis, IO = False):
    plane = vtk.vtkPlane()
    plane.SetOrigin(center)
    plane.SetNormal(axis)

    clipper = vtk.vtkClipDataSet()
    clipper.SetInputData(volume)
    clipper.SetClipFunction(plane)
    if IO:
        clipper.InsideOutOn()
    else:
        clipper.InsideOutOff()

    clipper.Update()

    return clipper.GetOutput()

def RotateVector(ray, normal, angle):
    angle = np.radians(angle)
    normal /= np.linalg.norm(normal)
    ray_rot = (ray*np.cos(angle) + np.cross(normal, ray) * np.sin(angle) + normal * np.dot(normal, ray) * (1 - np.cos(angle)))
    
    return ray_rot

def ProbeFilter(InputData, SourceData):
    probe = vtk.vtkProbeFilter()
    probe.AddInputData(InputData)
    probe.SetSourceData(SourceData)
    probe.Update()
    
    return probe.GetOutput()

def IncreaseResolution(surface, subdivisions = 3):
    subdivide = vtk.vtkLoopSubdivisionFilter()
    subdivide.SetInputData(surface)
    subdivide.SetNumberOfSubdivisions(subdivisions)
    subdivide.Update()

    return subdivide.GetOutput()

def Cylinder(CenterLine):
    tube_filter = vtk.vtkTubeFilter()
    tube_filter.SetInputData(CenterLine)
    tube_filter.SetRadius(Radius)
    tube_filter.SetNumberOfSides(2*NRaySection)
    tube_filter.CappingOff()
    tube_filter.Update()
    
    return tube_filter.GetOutput()

def Hemisphere(apex_centeroid, CL_axis):
    low_res = NRaySphere // 10
    
    sphere = vtk.vtkSphereSource()
    sphere.SetCenter(apex_centeroid)
    sphere.SetRadius(Radius)
    sphere.SetPhiResolution(low_res)
    sphere.SetThetaResolution(low_res)
    sphere.Update()

    Hemisphere = ClipSurfaceWPlane(sphere.GetOutput(), apex_centeroid, CL_axis)
    

    return Hemisphere

def fibonacci_sphere(samples=100):
    """Generates evenly distributed unit vectors over a sphere"""
    points = []
    phi = np.pi * (3. - np.sqrt(5.))  # Golden angle in radians

    for i in range(samples):
        y = 1 - (i / float(samples - 1)) * 2  # y goes from 1 to -1
        radius = np.sqrt(1 - y * y)  # Radius at given y
        theta = phi * i  # Golden angle increment

        x = np.cos(theta) * radius
        z = np.sin(theta) * radius
        points.append([x, y, z])

    return np.array(points)


In [4]:
SliceApex = ReadVTPFile(slice_apex)
SliceBase = ReadVTPFile(slice_base)

base_centeroid = GetCentroid(SliceBase)
apex_centeroid = GetCentroid(SliceApex)

AnnotationPoints = [base_centeroid, apex_centeroid]

centerline_axis = np.array([AnnotationPoints[1][0] - AnnotationPoints[0][0], 
                                AnnotationPoints[1][1] - AnnotationPoints[0][1], 
                                AnnotationPoints[1][2] - AnnotationPoints[0][2]])

CL_axis = centerline_axis/np.linalg.norm(centerline_axis)

point0 = np.array([AnnotationPoints[0][0] - CL_axis[0]*Base_Cover,
                    AnnotationPoints[0][1] - CL_axis[1]*Base_Cover, 
                    AnnotationPoints[0][2] - CL_axis[2]*Base_Cover])

point1 = np.array([AnnotationPoints[1][0] + CL_axis[0]*Apex_Cover,
                    AnnotationPoints[1][1] + CL_axis[1]*Apex_Cover, 
                    AnnotationPoints[1][2] + CL_axis[2]*Apex_Cover])


In [ ]:
print("--- Extracting the Centerline of the Myocardium")
cl_file = f"{InputFolder}/CenterLine.vtp"
CenterLine = Line(point0, point1, NSection)
WriteVTPFile(cl_file, BoldLine(CenterLine))

InputMBFBase = f"{InputFolder}/UpperMyo.vtu"
InputMBFApex = f"{InputFolder}/ApexMyo.vtu"

Myocardium = ReadVTPFile(InputMyocardium)
ConvertVTU = vtk.vtkAppendFilter()
ConvertVTU.SetInputData(Myocardium)
ConvertVTU.Update()
MyocardiumVTU = ConvertVTU.GetOutput()
UpperMyo = ClipVolumeWPlane(MyocardiumVTU, apex_centeroid, CL_axis, True)
ApexMyo = ClipVolumeWPlane(MyocardiumVTU, apex_centeroid, CL_axis, False)
WriteVTUFile(InputMBFBase, UpperMyo)
WriteVTUFile(InputMBFApex, ApexMyo)

InputMBFBase = f"{InputFolder}/UpperMBF.vtu"
InputMBFApex = f"{InputFolder}/ApexMBF.vtu"


MBF_ = ReadVTUFile(InputMBF)
MBF = ClipVolumeWPlane(MBF_, apex_centeroid, CL_axis, True)
Apex = ClipVolumeWPlane(MBF_, apex_centeroid, CL_axis, False)
WriteVTUFile(InputMBFBase, MBF)
WriteVTUFile(InputMBFApex, Apex)

#LV = ReadVTKFile(InputLV_binarylabelmap)

--- Extracting the Centerline of the Myocardium


In [30]:
def CastRaysVisualizations(CLPoints):
    
    #>>> Ray Casting across the Myocardium Cylinder 
    slices = vtk.vtkAppendPolyData()

    NRays_sample = 20
    ray_ = np.array([-CL_axis[2], 0, CL_axis[0]])
    res = 50
    angles = np.linspace(0, 360, NRays_sample, endpoint= False)
    Rays = vtk.vtkAppendPolyData()
    Intersection = vtk.vtkAppendPolyData()
    
    for k in range(0,len(CLPoints), int(len(CLPoints)/(NSection/10))):
        MBF_slice = SliceWPlane(MBF, CLPoints[k], CL_axis)
        Myo_slice = SliceWPlane(UpperMyo, CLPoints[k], CL_axis)
        slices.AddInputData(MBF_slice)

        origin = CLPoints[k]
        slice_rays = vtk.vtkAppendPolyData()
        for i in range(NRays_sample):
            ray_new = RotateVector(ray_, CL_axis, angles[i])
            point2 = np.array([origin[0]+ L_Section_Ray*ray_new[0], origin[1] + L_Section_Ray*ray_new[1], origin[2] + L_Section_Ray*ray_new[2]])
            ray = Line(origin, point2, res)
            ray_projected = ProbeFilter(ray, MBF)
            
            mbf_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("ImageScalars"))
            mbf_values = mbf_profile[mbf_profile > 0]
            average_mbf = np.mean(mbf_values)
            mbf_profile = numpy_to_vtk(np.array([average_mbf for _ in range(ray_projected.GetNumberOfPoints())]))
            
            mbf_profile.SetName("MBFProfile")
            

            territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
            territory = territory_profile[territory_profile > 0]
            
            if territory.size > 0:
                territory_tag = np.bincount(territory).argmax()
                territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
            else:
                territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
            
            territory_profile.SetName("TerritoryProfile")

            ray_projected.GetPointData().AddArray(mbf_profile)
            ray_projected.GetPointData().AddArray(territory_profile)

            intersection_ = vtk.vtkIntersectionPolyDataFilter()
            intersection_.SetInputData(0, Myo_slice)
            intersection_.SetInputData(1, ray_projected)
            intersection_.Update()
            Intersection.AddInputData(intersection_.GetOutput())

            '''
            LV_boundary = numpy_to_vtk(np.array([0 for _ in range(ray_projected.GetNumberOfPoints())]))
            LV_boundary.SetName("LVBoundaries")
            boundary_points = []
            Locator = vtk.vtkPointLocator()
            Locator.SetDataSet(ray_projected)
            Locator.BuildLocator()
            for i in range(Myo_slice.GetNumberOfPoints()):
                point = Myo_slice.GetPoint(i)
                closest_point_id = Locator.FindClosestPoint(point)

                if closest_point_id == -1:
                    continue
                else:
                    boundary_points.append(point)
                    LV_boundary.SetValue(closest_point_id, 1)

            ray_projected.GetPointData().AddArray(LV_boundary)
            '''


            slice_rays.AddInputData(BoldLine(ray_projected))
    
        slice_rays.Update()
    
        Rays.AddInputData(slice_rays.GetOutput())


    #>>> Ray Casting across the Apex Hemisphere
    NRays_sample = 100
    directions = fibonacci_sphere(NRays_sample)
    origin = apex_centeroid
    Rays_ = vtk.vtkAppendPolyData()

    for i in range(NRays_sample):
        ray_new = directions[i]
        point2 = np.array([origin[0]+ L_Sphere_Ray*ray_new[0], origin[1] + L_Sphere_Ray*ray_new[1], origin[2] + L_Sphere_Ray*ray_new[2]])
        ray = Line(origin, point2, res)
        ray_projected = ProbeFilter(ray, MBF)
            
        mbf_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("ImageScalars"))
        mbf_values = mbf_profile[mbf_profile > 0]
        average_mbf = np.mean(mbf_values)
        mbf_profile = numpy_to_vtk(np.array([average_mbf for _ in range(ray_projected.GetNumberOfPoints())]))
        
        mbf_profile.SetName("MBFProfile")
        

        territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
        territory = territory_profile[territory_profile > 0]
        
        if territory.size > 0:
            territory_tag = np.bincount(territory).argmax()
            territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
        else:
            territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
        
        territory_profile.SetName("TerritoryProfile")

        ray_projected.GetPointData().AddArray(mbf_profile)
        ray_projected.GetPointData().AddArray(territory_profile)

        intersection_ = vtk.vtkIntersectionPolyDataFilter()
        intersection_.SetInputData(0, ApexMyo)
        intersection_.SetInputData(1, ray_projected)
        intersection_.Update()
        Intersection.AddInputData(intersection_.GetOutput())

        '''
        LV_boundary = numpy_to_vtk(np.array([0 for _ in range(ray_projected.GetNumberOfPoints())]))
        LV_boundary.SetName("LVBoundaries")
        boundary_points = []
        Locator = vtk.vtkPointLocator()
        Locator.SetDataSet(ray_projected)
        Locator.BuildLocator()
        for i in range(Myo_slice.GetNumberOfPoints()):
            point = Myo_slice.GetPoint(i)
            closest_point_id = Locator.FindClosestPoint(point)

            if closest_point_id == -1:
                continue
            else:
                boundary_points.append(point)
                LV_boundary.SetValue(closest_point_id, 1)

        ray_projected.GetPointData().AddArray(LV_boundary)
        '''
        Rays_.AddInputData(BoldLine(ray_projected))


    Rays_.Update()
    Rays_ = ClipSurfaceWPlane(Rays_.GetOutput(), point1, CL_axis)
    Rays.AddInputData(Rays_)
    Rays.Update()
    ray_path = f"{InputFolder}/Rays.vtp"
    WriteVTPFile(ray_path, Rays.GetOutput())

    Intersection.Update()
    intersection_path = f"{InputFolder}/Intersection.vtp"
    WriteVTPFile(intersection_path, Intersection.GetOutput())

    slices.Update()
    slice_path = f"{InputFolder}/MyoSlices.vtp"
    WriteVTPFile(slice_path, slices.GetOutput())


In [ ]:
def RayCastingAlongMyocardium(CLPoints):

    NRays = NRaySection
    ray_ = np.array([-CL_axis[2], 0, CL_axis[0]])
    res = 50
    angles = np.linspace(0, 360, NRays, endpoint= False)
    Rays = vtk.vtkAppendPolyData()
    
    print("------ Ray Casting across Sections along Myocardium:")
    progress_old = -1
    for k in range(len(CLPoints)):
        progress_old = PrintProgress(k,len(CLPoints),progress_old)

        origin = CLPoints[k]
        slice_rays = vtk.vtkAppendPolyData()
        
        for i in range(NRays):
            ray_new = RotateVector(ray_, CL_axis, angles[i])
            point2 = np.array([origin[0]+ L_Section_Ray*ray_new[0], origin[1] + L_Section_Ray*ray_new[1], origin[2] + L_Section_Ray*ray_new[2]])
            ray = Line(origin, point2, res)
            ray_projected = ProbeFilter(ray, MBF)
            
            mbf_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("ImageScalars"))
            mbf_values = mbf_profile[mbf_profile > 0]
            average_mbf = np.mean(mbf_values)
            mbf_profile = numpy_to_vtk(np.array([average_mbf for _ in range(ray_projected.GetNumberOfPoints())]))
            
            mbf_profile.SetName("MBFProfile")
            

            territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
            territory = territory_profile[territory_profile > 0]
            
            if territory.size > 0:
                territory_tag = np.bincount(territory).argmax()
                territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
            else:
                territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
            
            territory_profile.SetName("TerritoryProfile")

            #ray_projected = ProbeFilter(ray, UpperMyo)
            ray_projected.GetPointData().AddArray(mbf_profile)
            ray_projected.GetPointData().AddArray(territory_profile)

            slice_rays.AddInputData(ray_projected)
        slice_rays.Update()
    
        Rays.AddInputData(slice_rays.GetOutput())
    
    Rays.Update()

    return Rays.GetOutput()

In [ ]:
def RayCastingAcrossSphere():
    directions = fibonacci_sphere(NRaySphere)
    origin = apex_centeroid
    Rays_Hemisphere = vtk.vtkAppendPolyData()
    res = 50

    print("------ Ray Casting across the Apex Hemisphere:")
    progress_old = -1
    for i in range(NRaySphere):
        progress = PrintProgress(i, NRaySphere, progress_old)
        progress_old = progress

        ray_new = directions[i]
        point2 = np.array([origin[0]+ L_Sphere_Ray*ray_new[0], origin[1] + L_Sphere_Ray*ray_new[1], origin[2] + L_Sphere_Ray*ray_new[2]])
        ray = Line(origin, point2, res)
        ray_projected = ProbeFilter(ray, MBF)
            
        mbf_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("ImageScalars"))
        mbf_values = mbf_profile[mbf_profile > 0]
        average_mbf = np.mean(mbf_values)
        mbf_profile = numpy_to_vtk(np.array([average_mbf for _ in range(ray_projected.GetNumberOfPoints())]))
        
        mbf_profile.SetName("MBFProfile")
        

        territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
        territory = territory_profile[territory_profile > 0]
        
        if territory.size > 0:
            territory_tag = np.bincount(territory).argmax()
            territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
        else:
            territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
        
        territory_profile.SetName("TerritoryProfile")

        #ray_projected = ProbeFilter(ray, UpperMyo)
        ray_projected.GetPointData().AddArray(mbf_profile)
        ray_projected.GetPointData().AddArray(territory_profile)

        Rays_Hemisphere.AddInputData(ray_projected)

    Rays_Hemisphere.Update()
    Rays_Hemisphere = ClipSurfaceWPlane(Rays_Hemisphere.GetOutput(), point1, CL_axis)

    return Rays_Hemisphere


In [137]:

def SpatialCorrespondance(Surface):
    with open(f"{InputFolder}/MBF_Territories_Labels.dat",'r') as infile:
        infile.readline()
        TerritoryLabels=[]
        TerritoryNames = ""
        for LINE in infile:
            line=LINE.split()
            for tag in TerritoryTag:
                if line[1].find(tag)>=0: 
                    TerritoryLabels.append(int(line[0]))
                    TerritoryNames += os.path.splitext(line[1])[0] + "+"
    TerritoryNames = TerritoryNames[:-1]

    ThresholdArray = np.zeros(Surface.GetNumberOfPoints())
    for i in range(Surface.GetNumberOfPoints()):
        if int(Surface.GetPointData().GetArray("TerritoryProfile").GetValue(i)) in TerritoryLabels:
            ThresholdArray[i] = 1
    
    ThresholdArrayVTK = numpy_to_vtk(ThresholdArray, deep=True)
    ThresholdArrayVTK.SetName(f"TerritoryLabels_{TerritoryNames}")
    Surface.GetPointData().AddArray(ThresholdArrayVTK)
    Surface.Modified()

    territory_vtu_ = vtk.vtkAppendFilter()
    territory_vtu_.AddInputData(Surface)
    territory_vtu_.Update()
    territory_vtu = territory_vtu_.GetOutput()
    TerritoryVTU = ThresholdByUpper(territory_vtu, f"TerritoryLabels_{TerritoryNames}", 1)
    TerritoryRegions = ExtractSurface(TerritoryVTU)
    WriteVTPFile(InputFolder + f"/Projected_Territories_{TerritoryNames}.vtp", TerritoryRegions)

    ischemic_vtu_ = vtk.vtkAppendFilter()
    ischemic_vtu_.AddInputData(Surface)
    ischemic_vtu_.Update()
    ischemic_vtu = ischemic_vtu_.GetOutput()
    IschemicVTU = ThresholdByUpper(ischemic_vtu, "IschemicProfile", 1)
    IschemicRegions = ExtractSurface(IschemicVTU)
    ischemic_surface_path = os.path.splitext(os.path.basename(InputIschemic))[0]
    WriteVTPFile(InputFolder + f"/{ischemic_surface_path}.vtp", IschemicRegions)

    TerritoryRegionsPoints=vtk_to_numpy(TerritoryRegions.GetPoints().GetData())
    IschemicRegionsPoints=vtk_to_numpy(IschemicRegions.GetPoints().GetData())

    TerritoryTesselationString=[]
    TerritoryIschemicString=[]
    for i in range(TerritoryRegions.GetNumberOfPoints()): TerritoryTesselationString.append(str(TerritoryRegionsPoints[i]))
    for i in range(IschemicRegions.GetNumberOfPoints()): TerritoryIschemicString.append(str(IschemicRegionsPoints[i]))
    TotalPointsString=TerritoryTesselationString+TerritoryIschemicString
    UniquePointsString=set(TotalPointsString)
    OverlapCounter=len(TotalPointsString)-len(UniquePointsString)

    DiceScore = (2*OverlapCounter)/(TerritoryRegions.GetNumberOfPoints() + IschemicRegions.GetNumberOfPoints())

    OverlapTag_Ischemic = np.zeros(IschemicRegions.GetNumberOfPoints())
    point_locator = vtk.vtkPointLocator()
    point_locator.SetDataSet(TerritoryRegions)
    point_locator.BuildLocator()

    tolerance = 0.01
    dist2 = vtk.reference(0.0)
    Mean_dist = 0
    counter = 0
    for i in range(IschemicRegions.GetNumberOfPoints()):
        point = IschemicRegions.GetPoint(i)
        closest_point_id = point_locator.FindClosestPointWithinRadius(tolerance, point, dist2)
        if closest_point_id != -1:
            OverlapTag_Ischemic[i] = 1
        else:
            counter += 1
            point_ = TerritoryRegions.GetPoint(i)
            Mean_dist += vtk.vtkMath.Distance2BetweenPoints(point, point_) ** 0.5
        
    Mean_dist /= counter

    return TerritoryRegions, IschemicRegions, DiceScore, Mean_dist, TerritoryNames


### Implementing the Method

In [31]:
print("--- Visualizing Ray Casting")
CLPoints = CenterLine.GetPoints()
CLPointsArray = np.array([CLPoints.GetPoint(i) for i in range(CLPoints.GetNumberOfPoints())])
CastRaysVisualizations(CLPointsArray)


--- Visualizing Ray Casting


/usr/local/anaconda3/envs/vtkfenics/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/anaconda3/envs/vtkfenics/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
2025-06-25 21:09:19.847 (95934.682s) [         1483757]            vtkMath.cxx:797   WARN| vtkMath::Jacobi: Error extracting eigenfunctions
2025-06-25 21:09:19.848 (95934.682s) [         1483757]            vtkMath.cxx:797   WARN| vtkMath::Jacobi: Error extracting eigenfunctions
2025-06-25 21:09:19.848 (95934.682s) [         1483757]    vtkPointLocator.cxx:868    ERR| vtkPointLocator (0x7f8cab19ac80): No points to subdivide
2025-06-25 21:09:19.848 (95934.682s) [         1483757]vtkIntersectionPolyData:2410  WARN| No Intersection between objects 
2025-06-25 21:09:19.855 (95934.689s) [         1483757]            vtkMath

## Verify Geometry and Coverage
Before proceeding, double-check the visuals:

 - Confirm that the centerline extends along the myocardium as expected.

 - Ensure that the rays generated from each centerline point adequately cover the myocardial wall in all directions.



In [139]:
print("--- Ray Casting Across the Myocardium")
Rays_Myocardium = RayCastingAlongMyocardium(CLPointsArray)

--- Ray Casting Across the Myocardium
------ Ray Casting across Sections along Myocardium:
    Progress: 0%
    Progress: 10%
    Progress: 20%
    Progress: 30%
    Progress: 40%
    Progress: 50%
    Progress: 60%
    Progress: 70%
    Progress: 80%
    Progress: 90%
    Progress: 100%


In [140]:
WriteVTPFile(f"{InputFolder}/Rays_Myocardium.vtp", Rays_Myocardium)

In [141]:
Rays_Hemisphere = RayCastingAcrossSphere()

------ Ray Casting across the Apex Hemisphere:
    Progress: 0%
    Progress: 10%
    Progress: 20%
    Progress: 30%
    Progress: 40%
    Progress: 50%
    Progress: 60%
    Progress: 70%
    Progress: 80%
    Progress: 90%
    Progress: 100%
    Progress: 100%


In [142]:
WriteVTPFile(f"{InputFolder}/Rays_Hemisphere.vtp", Rays_Hemisphere)

In [143]:
OutputSurface = vtk.vtkAppendPolyData()
OutputSurface.AddInputData(Rays_Myocardium)
OutputSurface.AddInputData(Rays_Hemisphere)
OutputSurface.Update()

OutputVolume_ = vtk.vtkAppendFilter()
OutputVolume_.AddInputData(OutputSurface.GetOutput())
OutputVolume_.Update()
OutputVolume = OutputVolume_.GetOutput()


In [144]:
MyocardiumSurface = ReadVTPFile(f"{InputFolder}/../MyocardiumSurface.vtp")


In [149]:
IschemicProfile_Array = vtk.vtkFloatArray()
IschemicProfile_Array.SetName("IschemicProfile")
IschemicProfile_Array.SetNumberOfComponents(1)
IschemicProfile_Array.SetNumberOfTuples(MyocardiumSurface.GetNumberOfPoints())

TerritoryProfile_Array = vtk.vtkFloatArray()
TerritoryProfile_Array.SetName("TerritoryProfile")
TerritoryProfile_Array.SetNumberOfComponents(1)
TerritoryProfile_Array.SetNumberOfTuples(MyocardiumSurface.GetNumberOfPoints())

Rays_Output = OutputVolume
Rays_ischemic = Rays_Output.GetPointData().GetArray("IschemicProfile")
Rays_territory = Rays_Output.GetPointData().GetArray("TerritoryProfile")

Locator = vtk.vtkPointLocator()
Locator.SetDataSet(Rays_Output)
Locator.BuildLocator()

tolerance = 1
dist2 = vtk.reference(0.0)


for i in range(MyocardiumSurface.GetNumberOfPoints()):
    point = MyocardiumSurface.GetPoint(i)
    closest_point_id = Locator.FindClosestPoint(point)
    
    
    #IschemicProfile_Array.SetValue(i, Rays_ischemic.GetValue(closest_point_id))
    #TerritoryProfile_Array.SetValue(i, Rays_territory.GetValue(closest_point_id))
    
    if closest_point_id == -1:
        IschemicProfile_Array.SetValue(i, 0)
        TerritoryProfile_Array.SetValue(i, -1)
    else:
        IschemicProfile_Array.SetValue(i, Rays_ischemic.GetValue(closest_point_id))
        TerritoryProfile_Array.SetValue(i, Rays_territory.GetValue(closest_point_id))
    

MyocardiumSurface.GetPointData().AddArray(IschemicProfile_Array)
MyocardiumSurface.GetPointData().AddArray(TerritoryProfile_Array)

WriteVTPFile(f"{InputFolder}/MyocardiumSurface.vtp", MyocardiumSurface)

In [150]:
TerritoryRegions, IschemicRegions, DiceScore, SpatialDifference, TerritoryNames = SpatialCorrespondance(MyocardiumSurface)
DiceScore = int(DiceScore*100)/100
print("Dice-score:", DiceScore)

if scale == 'cm':
    SpatialDifference = int(SpatialDifference*100)/100
elif scale == 'mm':
    SpatialDifference = int(SpatialDifference)/100
    
print("Spatial Difference between Non-overlapping points: ", SpatialDifference, "cm")


Dice-score: 0.74
Spatial Difference between Non-overlapping points:  13.27 cm


In [151]:
def SurfaceAreaCalculator(Surface):
    Mass = vtk.vtkMassProperties()
    Mass.AddInputData(Surface)
    Mass.Update()
    return Mass.GetSurfaceArea()

SurfaceArea = SurfaceAreaCalculator(MyocardiumSurface)

if scale == 'cm':
    SurfaceArea_Myocardium = int(SurfaceArea*100)/100
elif scale == 'mm':
    SurfaceArea_Myocardium = int(SurfaceArea)/100
    
print("Surface Area of the Myocardium: ", SurfaceArea_Myocardium, "cm^2")

Surface Area of the Myocardium:  169.08 cm^2


In [152]:
SurfaceArea = SurfaceAreaCalculator(TerritoryRegions)

if scale == 'cm':
    SurfaceArea_Territory = int(SurfaceArea*100)/100
elif scale == 'mm':
    SurfaceArea_Territory = int(SurfaceArea)/100

print("Territory Regions Surface Area:", SurfaceArea_Territory, "cm^2")


SurfaceArea = SurfaceAreaCalculator(IschemicRegions)

if scale == 'cm':
    SurfaceArea_Ischemic = int(SurfaceArea*100)/100
elif scale == 'mm':
    SurfaceArea_Ischemic = int(SurfaceArea)/100

print("Ischemic Regions Surface Area:", SurfaceArea_Ischemic, "cm^2")



Territory Regions Surface Area: 39.97 cm^2
Ischemic Regions Surface Area: 57.18 cm^2


In [153]:
ofile_path = InputFolder + "/RayBasedProfileResults.dat"
with open(ofile_path, 'w') as ofile:
    ofile.writelines(f"Selected Territory Regions: {TerritoryNames}\n")
    ofile.writelines(f"Myocardium Surface (cm\u00b2): {SurfaceArea_Myocardium}\n")
    ofile.writelines(f"Territory Regions Surface Area (cm\u00b2): {SurfaceArea_Territory}\n")
    ofile.writelines(f"Ischemic Regions Surface Area (cm\u00b2): {SurfaceArea_Ischemic}\n")
    ofile.writelines(f"Dice-Score: {DiceScore}\n")
    ofile.writelines(f"Spatial Difference (cm): {SpatialDifference}\n")